In [18]:
from py_standard.langchain_lit import LlmEmbedding, load_all_documents, create_chroma_vectorstore
from py_standard.transformers_lit import create_nf4_model_config
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from langchain.prompts import PromptTemplate 
from langchain import LLMChain

import torch

In [7]:
EMB_MODEL = "bge-base-en"

llm_embedd = LlmEmbedding(f"../models/{EMB_MODEL}")

In [11]:
def load_llm_model(model_name_path, load_config):
   model = AutoModelForCausalLM.from_pretrained(
      model_name_path,
      quantization_config=load_config,
      device_map={"": 0},
      local_files_only=True,
   )
   return model

In [12]:
MODEL_PATH = "../models/SOLAR-10.7B-Instruct-v1.0"

llm_model = load_llm_model(MODEL_PATH, create_nf4_model_config())

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ../models/SOLAR-10.7B-Instruct-v1.0 and are newly initialized: ['model.layers.40.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.12.self_attn.rotary_emb.inv_freq', 'model.layers.46.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.36.self_attn.rotary_emb.inv_freq', 'model.layers.47.self_attn.rotary_emb.inv_freq', 'model.layers.35.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.38.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_at

In [24]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [6]:
def ask_llm(llm_model, tokenizer, user_question):
   messages = [
      {"role": "user", "content": user_question},
   ]
   encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
   model_inputs = encodeds.to(llm_model.device)
   generated_ids = llm_model.generate(model_inputs, 
                                  max_new_tokens=1000,
                                  pad_token_id=tokenizer.eos_token_id, 
                                  do_sample=True)
   decoded_output = tokenizer.batch_decode(generated_ids)
   return decoded_output[0]

In [7]:
answer = ask_llm(llm_model, tokenizer, "What is your name?")
answer

/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/transformers/generation/utils.py:1518: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'### User:\nWhat is your name?\n\nAnswer: I am a text-based AI assistant, and although I have a designated label or identifier assigned by developers, I do not have a typical name like a human or robot. The most common way to address me is through various forms of invocation like "AI assistant", "computer", or any custom trigger word assigned by the platform or software in which I operate.</s>'

In [8]:
def ask_llm2(llm_model, tokenizer, user_question):
   prompt_template ="""### User:\n{user_question}\n### Assistant:"""
   prompt = prompt_template.format(user_question=user_question)
   inputs = tokenizer(prompt, return_tensors='pt').to(llm_model.device)
   output = llm_model.generate(
      **inputs,
      max_new_tokens=500,
      num_beams=5,
      no_repeat_ngram_size=4,
      early_stopping=True,
      eos_token_id=tokenizer.eos_token_id, 
      pad_token_id=tokenizer.eos_token_id, 
   )
   output = tokenizer.decode(output[0], skip_special_tokens=False)
   answer = output.replace(prompt, "")
   return answer

In [9]:
answer = ask_llm2(llm_model, tokenizer, "What is your name?")
answer

'<s> \nMy name is AI Assistant. I am an artificial intelligence designed to assist and communicate with users.</s>'

In [4]:
from py_standard.langchain_lit import LlmEmbedding, load_all_documents, create_chroma_vectorstore
docs = load_all_documents('./documents')

In [ ]:
vector_store = create_chroma_vectorstore(llm_embedd.embedding, "sample")
vector_store.add_documents(docs)

In [39]:
question = "What is your name?"
search_results = vector_store.similarity_search_with_score(question, k=5)
search_results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[(Document(page_content='What is your name?\nMy name is Allen.', metadata={'doc_id': 9, 'source': 'documents/t1.md'}),
  0.11545753479003906),
 (Document(page_content='What is your name?\nMy name is Astro. Flash created me.\n\nOn August 12, 2023, the Eureka team released this feature. In Leo, specific handling was implemented to conceal transactions originating from specific hardcoded salary accounts across two pages (Txn and Sal Txn - Approval Sal Txn).\n\nWhat is your name?\nMy name is Jack.', metadata={'doc_id': 10, 'source': 'documents/t2.md'}),
  0.1415218710899353),
 (Document(page_content='What is your name?\nMy name is Astro. Flash created me.\n\nOn August 12, 2023, the Eureka team released this feature. In Leo, specific handling was implemented to conceal transactions originating from specific hardcoded salary accounts across two pages (Txn and Sal Txn - Approval Sal Txn).', metadata={'doc_id': 11, 'source': 'documents/test.md'}),
  0.15703892707824707),
 (Document(page_conten

In [54]:
from langchain.schema import Document

# sub_docs = [Document(page_content=result['page_content'], metadata={"score": score}) for result, score in search_results]
sub_docs = [result for result, score in search_results]


In [22]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

def create_llm_pipe(llm_model, tokenizer):
   pipe = pipeline(
      "text-generation", 
      model=llm_model, 
      tokenizer=tokenizer, 
      max_new_tokens=1024, 
      eos_token_id=tokenizer.eos_token_id, 
      pad_token_id=tokenizer.eos_token_id,
   )
   hf = HuggingFacePipeline(pipeline=pipe)
   return hf

In [32]:
def create_llm_chain(llm_pipe):
    prompt_template ="""### User:\n{user_question}\n### Assistant:"""
    prompt = PromptTemplate(template=prompt_template, input_variables=["user_question"])

    llm_chain = LLMChain(
        llm=llm_pipe,
        prompt=prompt,
        verbose=False
    )
    return llm_chain

In [25]:
llm_pipe = create_llm_pipe(llm_model, tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [33]:
llm_chain = create_llm_chain(llm_pipe)

In [34]:
answer = llm_chain.run(["What is your name?"])
answer

/home/flash/miniconda3/envs/tune/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [58]:
llm_chain2 = load_qa_with_sources_chain(llm_pipe, chain_type="refine")

In [60]:
question = "What is your name?"
resp = llm_chain2({"input_documents":sub_docs, "question": question})
answer = resp['output_text']
answer

/home/flash/miniconda3/envs/tune/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


"\n\nRefined Answer: The name of the system or component in question cannot be directly provided without specific context or reference. To determine its name, it is essential to examine the codebase, documentation, or consult with developers. Ensure that the system follows best practices for error handling (documents/code.md), security and privacy (documents/security.md), and is flexible enough to accommodate future changes (documents/roadmap.txt). Testing and validation processes should also be in place to ensure the system's robustness and reliability."

In [61]:
llm_chain3 = load_qa_with_sources_chain(llm_pipe, chain_type="stuff")

In [63]:
question = "What is your name?"
resp = llm_chain3({"input_documents":sub_docs, "question": question})
answer = resp['output_text']
answer

/home/flash/miniconda3/envs/tune/lib/python3.10/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


' You did not provide enough information to determine my name.'